<a href="https://colab.research.google.com/github/shivams289/Projects/blob/main/Portfolio_allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import random

In [ ]:
#ann_vol - annualized volatatilty = periodic_volatailty*sqrt(T)
def ann_vol(r, periods_per_year):
  return r.std()*(periods_per_year**0.5)


def ann_ret(r, periods_per_year):
  compound_growth = (1+r).prod()                          # This will multiply every (1+ return_row)
  n_periods = r.shape[0]                                  # Given by number of rows that our return dataframe have
  return compound_growth**(periods_per_year/n_periods)-1  # (1+r)**(T/n) = compound_growth  => 

                      #rfr - risk_free_rate
                      #r - return


def sharpe_ratio(r, rfr, periods_per_year):
  rf_per_period = (1+rfr)**(1/periods_per_year)-1
  excess_return = r - rf_per_period
  ann_excess_return = ann_ret(excess_return, periods_per_year)
  ann_vol = ann_vol(excess_return, periods_per_year)
  
  return ann_excess_return/n_vol



In [ ]:
from scipy.optimize import minimize

#minimizing volatility

def minimize_vol(target_return, er, cov):
  n = er.shape[0]
  init_guess = np.repeat(1/n,n)
  bounds = ((0.0, 1.0), )*n

  return_is_target ={
      'type':'eq',
      'args':(er,),
      'fun':lambda weights, er: target_return - portfolio_return(weights, er)
  }

  weights_sum_to_1 ={
      'type':'eq',
      'fun':lambda weights:np.sum(weights)-1
  }

  results = minimize(
      portfolio_vol, init_guess, args = (cov, ), method = 'SLSQP'
      options = {'disp':False}
      constraints = (return_is_target, weights_sum_to_1)
      bounds = bounds
  )

  return results.x

In [ ]:
def optimal_weights(n_ponts, er, cov):
  target_returns = np.linspace(er.min(), er.max(), n_points)
  weights = [minimize_vol(target_return, er, cov) for target_return in target_returns]
  return weights  

In [ ]:
def max_sharpe_ratio(rfr, er, cov):
  n = er.shape[0]
  init_guess = np.repeat(1/n,n)
  bounds = ((0.0, 1.0), )*n
  weights_sum_to_1 ={
      'type':'eq',
      'fun':lambda weights:np.sum(weights)-1
  }

  def neg_sharpe(weights, rfr, er, cov):
    r = portfolio_return(weights, er)
    vol = portfolio_vol(weights, cov)
    return -(r-rfr)/vol

  weights = minimize(neg_sharpe, init_guess, args = (rfr, er, cov), method = 'SLSQP', options = {'disp':False}, 
                     constraints  = (weights_sum_to_1,), bounds = bounds)
  
  return weights.x

